<a href="https://colab.research.google.com/github/Remo6985/AOL-AI/blob/main/AOL_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
from sklearn.model_selection import KFold
import os
import shutil
from glob import glob
import numpy as np

In [3]:
# Mount Google Drive
drive.mount('/content/gdrive')

# Unzip dataset
!scp '/content/gdrive/My Drive/AOL AI/AOL AI.zip' '/content/AOL AI.zip'
!unzip '/content/AOL AI.zip' -d '/content/'

Mounted at /content/gdrive
Archive:  /content/AOL AI.zip
   creating: /content/AOL AI/
   creating: /content/AOL AI/test/
   creating: /content/AOL AI/test/Ectomorph/
  inflating: /content/AOL AI/test/Ectomorph/-User-00-Ectomorph-10-_png_jpg.rf.4b7f0eeacfb7aa53635d2becbfd24bab.jpg  
  inflating: /content/AOL AI/test/Ectomorph/-User-00-Ectomorph-12-_jpg.rf.51b76c781df0ab9a3724360f18c3181d.jpg  
  inflating: /content/AOL AI/test/Ectomorph/-User-00-Ectomorph-21-_png_jpg.rf.a50c234bac2ed5a606ded7ef3a1710fe.jpg  
  inflating: /content/AOL AI/test/Ectomorph/-User-00-Ectomorph-24-_png_jpg.rf.14e0d93788df08bc3d721035be89cc11.jpg  
  inflating: /content/AOL AI/test/Ectomorph/-User-00-Ectomorph-25-_png_jpg.rf.9f54d4001e4be7476b0c7a1d6ff60dcb.jpg  
  inflating: /content/AOL AI/test/Ectomorph/-User-00-Ectomorph-27-_png_jpg.rf.a3aadc437f2e672be4cd6a3784f65888.jpg  
  inflating: /content/AOL AI/test/Ectomorph/-User-00-Ectomorph-29-_png_jpg.rf.73922fa2cc3dc9500de2493246c02cf5.jpg  
  inflating: /cont

In [3]:
# Install YOLOv8
!pip install ultralytics

from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 901.3/901.3 kB 14.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# Define dataset path
dataset_path = "/content/AOL AI/"
num_folds = 3  # Number of folds for cross-validation

# Gather all class folders from train/
class_folders = [os.path.join(dataset_path, "train", class_dir) for class_dir in os.listdir(os.path.join(dataset_path, "train")) if os.path.isdir(os.path.join(dataset_path, "train", class_dir))]

# Gather all images from train/
all_images = []
for class_folder in class_folders:
    class_images = glob(os.path.join(class_folder, "*"))  # All images in this class folder
    all_images.extend(class_images)

print(f"Total images: {len(all_images)}")
print(f"Classes: {[os.path.basename(folder) for folder in class_folders]}")

Total images: 950
Classes: ['Ectomorph', 'Endomorph', 'Mesomorph']


In [6]:
# Set up K-Fold Cross-Validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

fold = 1
fold_accuracies = []  # To store accuracy for each fold

In [9]:
for train_idx, val_idx in kf.split(all_images):
    print(f"Processing Fold {fold}...")

    # Create directories for this fold
    fold_dir = f"/content/AOL_AI_fold_{fold}"
    train_dir = os.path.join(fold_dir, "train")
    val_dir = os.path.join(fold_dir, "val")
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)

    # Split data into train and validation sets
    train_images = [all_images[i] for i in train_idx]
    val_images = [all_images[i] for i in val_idx]

    # Move train images to train/ preserving class structure
    for img in train_images:
        class_name = os.path.basename(os.path.dirname(img))  # Extract class name from folder structure
        class_train_dir = os.path.join(train_dir, class_name)
        os.makedirs(class_train_dir, exist_ok=True)
        shutil.copy(img, class_train_dir)

    # Move validation images to val/ preserving class structure
    for img in val_images:
        class_name = os.path.basename(os.path.dirname(img))  # Extract class name from folder structure
        class_val_dir = os.path.join(val_dir, class_name)
        os.makedirs(class_val_dir, exist_ok=True)
        shutil.copy(img, class_val_dir)

    # Train YOLOv8 Model
    model = YOLO("yolov8n-cls.pt")  # Classification model
    fold_results = model.train(
        data='/content/AOL AI/',  # Pass the YAML file path
        epochs=100,
        imgsz=640,
        batch=16,
        patience=10,  # Early stopping
        optimizer="SGD",
        lr0=0.01,
        weight_decay=1e-4,
        augment=True,
    )

    # Extract accuracy for the current fold
    accuracy = fold_results.top1  # Replace with the correct metric key
    fold_accuracies.append(accuracy)

    print(f"Fold {fold} Accuracy: {accuracy:.2f}")
    fold += 1


Processing Fold 1...
Ultralytics 8.3.51 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=/content/AOL AI/, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=

train: Scanning /content/AOL AI/train... 950 images, 0 corrupt: 100%|██████████| 950/950 [00:00<?, ?it/s]
val: Scanning /content/AOL AI/val... 31 images, 0 corrupt: 100%|██████████| 31/31 [00:00<?, ?it/s]


optimizer: SGD(lr=0.01, momentum=0.937) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0001), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/classify/train2
Starting training for 100 epochs...

      Epoch    GPU_mem       loss  Instances       Size


      1/100      1.65G      1.066          6        640: 100%|██████████| 60/60 [00:36<00:00,  1.64it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00, 15.18it/s]

                   all      0.387          1


...


22 epochs completed in 0.245 hours.
Optimizer stripped from runs/classify/train4/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train4/weights/best.pt, 3.0MB

Validating runs/classify/train4/weights/best.pt...
Ultralytics 8.3.51 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1,438,723 parameters, 0 gradients, 3.3 GFLOPs
train: /content/AOL AI/train... found 950 images in 3 classes ✅ 
val: /content/AOL AI/val... found 31 images in 3 classes ✅ 
test: /content/AOL AI/test... found 244 images in 3 classes ✅ 


               classes   top1_acc   top5_acc:   0%|          | 0/1 [00:00<?, ?it/s]

WARNING ⚠️ ClassificationModel does not support 'augment=True' prediction. Reverting to single-scale prediction.


               classes   top1_acc   top5_acc: 100%|██████████| 1/1 [00:00<00:00, 10.84it/s]


                   all      0.903          1
Speed: 0.8ms preprocess, 1.9ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train4
Fold 3 Accuracy: 0.90


In [11]:
# Compute mean and standard deviation of accuracies
mean_accuracy = np.mean(fold_accuracies)
std_accuracy = np.std(fold_accuracies)

# Print aggregated results
print(f"Cross-Validation Accuracies: {fold_accuracies}")
print(f"Mean Accuracy: {mean_accuracy:.2f}")
print(f"Standard Deviation: {std_accuracy:.2f}")


Cross-Validation Accuracies: [0.9032257795333862, 0.9032257795333862, 0.9032257795333862]
Mean Accuracy: 0.90
Standard Deviation: 0.00


In [12]:
!scp -r /content/runs '/content/gdrive/My Drive/AOL AI'